In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from datetime import datetime, timedelta
import pandas as pd

current_date = pd.to_datetime(datetime.utcnow()).floor('H') # - timedelta(hours=1)
print(f'{current_date=}')
# current_date = pd.Timestamp('2023-02-28 09:00:00')

current_date=Timestamp('2023-02-28 09:00:00')


In [3]:
from src.inference import load_batch_of_features_from_store

features = load_batch_of_features_from_store(current_date)

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/12447


Connected. Call `.close()` to terminate connection gracefully.
Fetching data from 2023-01-31 09:00:00 to 2023-02-28 08:00:00


2023-02-28 11:54:57,000 INFO: USE `taxi_demand_featurestore`
2023-02-28 11:54:57,511 INFO: SELECT `fg0`.`pickup_hour` `pickup_hour`, `fg0`.`rides` `rides`, `fg0`.`pickup_location_id` `pickup_location_id`
FROM `taxi_demand_featurestore`.`time_series_hourly_feature_group_1` `fg0`
WHERE `fg0`.`pickup_hour` >= TIMESTAMP '2023-01-30 09:00:00.000' AND `fg0`.`pickup_hour` <= TIMESTAMP '2023-03-01 08:00:00.000'


ts_data=               pickup_hour  rides  pickup_location_id
6523   2023-01-31 09:00:00      0                   1
6815   2023-01-31 10:00:00      0                   1
6979   2023-01-31 11:00:00      0                   1
7263   2023-01-31 12:00:00      3                   1
7647   2023-01-31 13:00:00      0                   1
...                    ...    ...                 ...
183150 2023-02-28 04:00:00      0                 265
183455 2023-02-28 05:00:00      3                 265
183677 2023-02-28 06:00:00     14                 265
183919 2023-02-28 07:00:00     17                 265
184298 2023-02-28 08:00:00     16                 265

[178080 rows x 3 columns]


In [5]:
from src.inference import (
    load_model_from_registry,
    get_model_predictions
)

model = load_model_from_registry()
predictions = get_model_predictions(model, features)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/12447
Connected. Call `.close()` to terminate connection gracefully.

In [7]:
predictions['pickup_hour'] = current_date
predictions

,pickup_location_id,predicted_demand,pickup_hour
0,264,48.0,2023-02-28 09:00:00
1,91,0.0,2023-02-28 09:00:00
2,97,2.0,2023-02-28 09:00:00
3,231,84.0,2023-02-28 09:00:00
4,165,1.0,2023-02-28 09:00:00
...,...,...,...
260,196,0.0,2023-02-28 09:00:00
261,203,0.0,2023-02-28 09:00:00
262,63,0.0,2023-02-28 09:00:00
263,190,0.0,2023-02-28 09:00:00


### Save these predictions in the feature store, so they can be later consumed by our Streamlit app

In [9]:
from src.feature_store_api import get_feature_store
import src.config as config

# connect to the feature group
feature_group = get_feature_store().get_or_create_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTIONS,
    version=1,
    description="Predictions generate by our production model",
    primary_key = ['pickup_location_id', 'pickup_hour'],
    event_time='pickup_hour',
)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/12447
Connected. Call `.close()` to terminate connection gracefully.


In [10]:
feature_group.insert(predictions, write_options={"wait_for_job": False})

Uploading Dataframe: 0.00% |          | Rows 0/265 | Elapsed Time: 00:00 | Remaining Time: ?

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/12447/jobs/named/model_predictions_feature_group_1_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x131826b50>, None)